In [1]:
import geopandas as gpd
ds=gpd.read_file("DS24_Current season_TH_test.geojson").to_crs(epsg=4236).copy()
bbox = ds.total_bounds
ds=ds[["Farm id", "Crop", "ns1:coordinates"]]
ds=ds.set_index("Farm id")
ds.loc[:, ["Crop"]]
ds

,Crop,ns1:coordinates
Farm id,,
DO4C13S05A,Corn,"101.87851139,16.60973811000001 101.87860395,16..."
DO4C15S02A,Corn,"102.02565554,16.54022877 102.02583757,16.54072..."
DO4C15S04A,Corn,"102.02388005000002,16.543836260000006 102.0231..."
DO4C16S01A,Corn,"101.97520326,16.527570659999995 101.97509415,1..."
DO4C16S02A,Corn,"101.97278114,16.526720929999996 101.97310569,1..."
...,...,...
WN4C30S28C,Corn,"102.01263402,16.465605699999998 102.01252003,1..."
WN4C34S14B,Corn,"102.02239302,16.510191529999986 102.02235624,1..."
WN4C15S01B,Corn,"102.04262878999998,16.469971639999997 102.0432..."


In [35]:
import pystac_client
api = pystac_client.Client.open("https://stactest.eofactory.ai")

search_2023 = api.search(
    datetime=['2023-05','2024-02'],
    collections=["sentinel-1-rtc-mosaic-karnataka"], ##### tí đổi
    max_items=25
)
items = search_2023.item_collection()
items

In [37]:
#Data
import numpy as np
import stackstac
import rioxarray
chunk_size = 2048
# signed_items = [pc.sign(item).to_dict() for item in items]
# dict_items = [item.to_dict() for item in items]
data = stackstac.stack(
        items,
        bounds_latlon=bbox,
        dtype='float32',
        assets=["VH"],  # vh only
        chunksize=chunk_size,
        resolution=10,
        epsg=32647,
    ).where(lambda x: x > -32768, other=np.nan)  # sentinel-1-rtc uses -32768 as nodata
    # .assign_coords(band=lambda x: x.common_name.rename("band"))  # use common names

data

<xarray.DataArray 'stackstac-54025fb3207ad6553567bc4eef5dbd7c' (time: 27,
                                                                band: 1,
                                                                y: 22788,
                                                                x: 12366)>
dask.array<where, shape=(27, 1, 22788, 12366), dtype=float32, chunksize=(1, 1, 2048, 2048), chunktype=numpy.ndarray>
Coordinates: (12/16)
  * time            (time) datetime64[ns] 2022-12-30 2023-01-11 ... 2023-11-07
    id              (time) <U24 'mosaiced_20221230T000000' ... 'mosaiced_2023...
  * band            (band) <U2 'VH'
  * x               (x) float64 2.298e+05 2.299e+05 ... 3.535e+05 3.535e+05
  * y               (y) float64 2.064e+06 2.064e+06 ... 1.836e+06 1.836e+06
    platform        <U10 'Sentinel-1'
    ...              ...
    start_datetime  (time) <U20 '2022-12-30T00:00:00Z' ... '2023-11-07T00:00:...
    proj:centroid   (time) object {'lat': 1686660.0642546741, 'lon': 20754.89...
    instruments     <U3 'MSI'
    raster:bands    object set()
    title           <U41 'VH: vertical transmit, horizontal receive'
    epsg            int64 32644
Attributes:
    spec:        RasterSpec(epsg=32644, bounds=(229850, 1836260, 353510, 2064...
    crs:         epsg:32644
    transform:   | 10.00, 0.00, 229850.00|\n| 0.00,-10.00, 2064140.00|\n| 0.0...
    resolution:  10

In [17]:
# %%time
# import pandas as pd
# df=pd.read_csv("farm-ds-final.csv")
# df


CPU times: user 124 ms, sys: 28 ms, total: 152 ms
Wall time: 152 ms


,id,2022-11-04,2022-11-16,2022-11-28,2022-12-10,2022-12-22,2023-01-03,2023-01-15,2023-01-27,2023-02-08,...,2023-05-27,2023-06-08,2023-06-20,2023-07-02,2023-07-14,2023-07-26,2023-08-07,2023-08-19,2023-08-31,2023-09-12
0,1,-12.419966,-14.718754,-16.567192,-13.321877,-13.797693,-15.072689,-14.183838,-10.869824,-15.708220,...,-15.138991,-18.097258,-17.526844,-17.815108,-16.575856,-16.112016,-19.522162,-12.447709,-14.389368,-14.573981
1,1,-12.002182,-15.012148,-18.877948,-12.444623,-12.614114,-13.418167,-14.196879,-11.689258,-15.535236,...,-14.972300,-19.656938,-18.554854,-17.341280,-16.905343,-15.499929,-20.099711,-15.086059,-14.342763,-15.248909
2,1,-12.277605,-14.663686,-15.402168,-12.673289,-14.294425,-13.264031,-13.765332,-12.871502,-16.275170,...,-15.034176,-18.611599,-16.837963,-18.999377,-17.007025,-15.640580,-19.530524,-17.314536,-16.303362,-16.792685
3,1,-13.617784,-14.957466,-14.810104,-14.242361,-13.155283,-15.320282,-12.323627,-13.409324,-16.719618,...,-15.637730,-18.354724,-16.447213,-19.684951,-17.875694,-16.687563,-18.345470,-16.158528,-17.957877,-15.952545
4,1,-15.681765,-16.456573,-14.997072,-15.134715,-13.431351,-15.610487,-12.814217,-14.572465,-15.710149,...,-16.860933,-18.379941,-18.828548,-18.395018,-18.152016,-18.081023,-17.672600,-13.362275,-16.850674,-15.608144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61668,827,-14.999800,-17.180566,-16.131660,-17.013206,-20.958580,-18.604133,-18.908203,-20.082518,-20.891396,...,-17.664536,-18.065533,-17.196645,-15.548138,-17.133901,-12.306667,-14.800411,-13.338131,-14.614765,-14.529509
61669,827,-14.785804,-16.986782,-15.870174,-20.478685,-19.504435,-17.961060,-19.084439,-20.947378,-21.762452,...,-18.530790,-20.274156,-18.456304,-16.468211,-18.570214,-13.268043,-16.845107,-13.923823,-16.748332,-16.432421
61670,827,-16.302536,-15.741450,-15.948386,-18.112725,-17.780991,-20.428281,-16.774863,-16.584113,-18.803585,...,-18.511919,-17.473254,-17.520242,-16.823756,-16.257167,-16.581083,-15.064447,-12.537356,-14.101117,-16.526393
61671,827,-16.266315,-16.386153,-16.549602,-17.734884,-18.718146,-21.321985,-17.613977,-18.715269,-19.881741,...,-19.108031,-18.457133,-18.976761,-18.677301,-16.896032,-13.536183,-14.834338,-13.611258,-14.649106,-18.403781


In [38]:
epsg=items[0].properties['proj:epsg']
df_epsg=ds.to_crs(epsg=epsg).copy()
df_epsg


,crop_type,geometry
id,,
1,Corn,"POLYGON ((324272.594 2015385.320, 324293.501 2..."
2,Rice,"POLYGON ((347419.990 2035367.031, 347608.004 2..."
3,Rice,"POLYGON ((347524.504 2035769.730, 347532.002 2..."
4,Corn,"POLYGON ((323559.122 2014344.406, 323607.324 2..."
5,Rice,"POLYGON ((344339.136 2020294.864, 344343.015 2..."
...,...,...
823,Corn,"POLYGON ((349862.393 2023551.038, 349881.648 2..."
824,Corn,"POLYGON ((344479.660 2027672.754, 344509.007 2..."
825,Corn,"POLYGON ((345663.434 2028275.507, 345490.448 2..."


In [39]:
import warnings
warnings.filterwarnings('ignore')
def farmprocessing(r):
        
    import rioxarray
    farm=r['ns1:coordinates']
    bbox = farm.bounds
    try:
        data = stackstac.stack(
            items,
            bounds=bbox,
            dtype='float32',
            assets=["VH"],  # vh only
            chunksize=chunk_size,
            resolution=10,
            epsg=32647
        ).where(lambda x: x > -32768, other=np.nan)  # sentinel-1-rtc uses -32768 as nodata
        # .assign_coords(band=lambda x: x.common_name.rename("band"))  # use common names
        cliped = data.rio.clip([farm], all_touched=False)
        
        cliped=10*np.log10(cliped) # convert to deciband
        tmp=cliped.to_numpy().squeeze(axis=1)
        times, height, width=tmp.shape
        tmp1=tmp.reshape(times, height*width).T
        tmp1=tmp1[~np.isnan(tmp1).all(axis=1),:]
        return tmp1
    except:
        return []

In [40]:
%%time
df2= df_epsg.apply(farmprocessing, axis=1)

CPU times: user 5min 19s, sys: 52 s, total: 6min 11s
Wall time: 7min 26s


In [41]:
import datetime
times = np.datetime_as_string(data.time.data, unit='D').tolist()
times.append("id")

In [42]:
times

['2022-12-30',
 '2023-01-11',
 '2023-01-23',
 '2023-02-04',
 '2023-02-16',
 '2023-02-28',
 '2023-03-12',
 '2023-03-24',
 '2023-04-05',
 '2023-04-17',
 '2023-04-29',
 '2023-05-11',
 '2023-05-23',
 '2023-06-04',
 '2023-06-16',
 '2023-06-28',
 '2023-07-10',
 '2023-07-22',
 '2023-08-03',
 '2023-08-15',
 '2023-08-27',
 '2023-09-08',
 '2023-09-20',
 '2023-10-02',
 '2023-10-14',
 '2023-10-26',
 '2023-11-07',
 'id']

In [43]:
arr = []
for index, value in df2.items():
    for item in value:
        try:
            it = item.tolist()
            it.append(index)
            arr.append(it)
        except:
            print(item)
import pandas as pd 
df = pd.DataFrame(arr, columns=times)
df.merge(ds, how='left', left_on='id', right_index=True)
df.to_csv("tranning_data.csv")
df

,2022-12-30,2023-01-11,2023-01-23,2023-02-04,2023-02-16,2023-02-28,2023-03-12,2023-03-24,2023-04-05,2023-04-17,...,2023-08-03,2023-08-15,2023-08-27,2023-09-08,2023-09-20,2023-10-02,2023-10-14,2023-10-26,2023-11-07,id
0,-8.578745,-8.017080,-8.095535,-7.869020,-7.796859,-7.318550,-8.283512,-5.615054,-8.827310,-8.761256,...,-9.279830,-7.133316,-8.513609,-8.683581,-6.659398,-8.752501,-6.717196,-7.058621,-7.979610,1
1,-9.206903,-7.775376,-7.317757,-8.621891,-8.366980,-7.641796,-7.323046,-6.204970,-8.763176,-8.930828,...,-9.644441,-6.631653,-9.249391,-9.508042,-7.047287,-8.067330,-6.389507,-7.138859,-8.441739,1
2,-9.451531,-7.377574,-6.041294,-8.921891,-7.958169,-7.838725,-6.685086,-7.308954,-8.853135,-10.180252,...,-10.612381,-5.986076,-9.388578,-7.470451,-7.298187,-6.735642,-6.018382,-5.776673,-8.917217,1
3,-7.875578,-7.641963,-5.437450,-7.726506,-8.035592,-8.039540,-6.486625,-7.363543,-8.509148,-11.173606,...,-11.832764,-6.377018,-7.483518,-7.928420,-6.892670,-6.053260,-6.119555,-5.187320,-9.118670,1
4,-6.438655,-7.499561,-6.027222,-8.095627,-7.663611,-8.743983,-7.346660,-6.983235,-9.631377,-11.657347,...,-11.424780,-8.226938,-7.392450,-9.938508,-7.073982,-6.778762,-6.189435,-6.242713,-8.558163,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61754,-8.919444,-8.977633,-11.200722,-9.596588,-9.692334,-10.160093,-8.727581,-9.155390,-8.692006,-6.806371,...,-7.233694,-7.585678,-9.066571,-7.356014,-8.539863,-9.637972,-9.004229,-8.770036,-9.154476,827
61755,-9.831942,-9.204880,-10.451384,-11.644909,-9.357404,-11.594908,-10.035514,-9.287397,-10.120270,-8.450119,...,-7.404220,-7.830167,-7.524363,-7.003328,-9.849733,-8.943263,-10.166101,-8.884039,-11.896362,827
61756,-8.372482,-7.837086,-10.661010,-8.588747,-10.366450,-8.134382,-9.881011,-8.899261,-8.712926,-7.248343,...,-8.430062,-7.521735,-9.666697,-8.092477,-7.888774,-7.772822,-7.364169,-11.367187,-7.333651,827
61757,-9.249858,-8.167767,-9.389226,-9.496934,-9.675346,-9.457351,-9.877887,-9.535177,-9.977454,-8.401375,...,-7.734263,-7.088699,-8.959745,-8.170175,-8.660059,-7.902033,-8.509414,-10.264296,-9.479219,827


In [27]:
def build_data(r):
    farm = r["geometry"]
    cliped = data.rio.clip([farm], all_touched=False)
    cliped=10*np.log10(cliped)
    tmp=cliped.to_numpy().squeeze(axis=1)
    times, height, width=tmp.shape
    tmp1=tmp.reshape(times, height*width).T
    tmp1=tmp1[~np.isnan(tmp1).all(axis=1),:]

In [46]:
trainingds=pd.merge(df, ds, how="left", left_on="id", right_index=True)
trainingds.head()

,2022-12-30,2023-01-11,2023-01-23,2023-02-04,2023-02-16,2023-02-28,2023-03-12,2023-03-24,2023-04-05,2023-04-17,...,2023-08-27,2023-09-08,2023-09-20,2023-10-02,2023-10-14,2023-10-26,2023-11-07,id,crop_type,geometry
0,-8.578745,-8.017080,-8.095535,-7.869020,-7.796859,-7.318550,-8.283512,-5.615054,-8.827310,-8.761256,...,-8.513609,-8.683581,-6.659398,-8.752501,-6.717196,-7.058621,-7.979610,1,Corn,"POLYGON ((79.33320 18.22094, 79.33339 18.22107..."
1,-9.206903,-7.775376,-7.317757,-8.621891,-8.366980,-7.641796,-7.323046,-6.204970,-8.763176,-8.930828,...,-9.249391,-9.508042,-7.047287,-8.067330,-6.389507,-7.138859,-8.441739,1,Corn,"POLYGON ((79.33320 18.22094, 79.33339 18.22107..."
2,-9.451531,-7.377574,-6.041294,-8.921891,-7.958169,-7.838725,-6.685086,-7.308954,-8.853135,-10.180252,...,-9.388578,-7.470451,-7.298187,-6.735642,-6.018382,-5.776673,-8.917217,1,Corn,"POLYGON ((79.33320 18.22094, 79.33339 18.22107..."
3,-7.875578,-7.641963,-5.437450,-7.726506,-8.035592,-8.039540,-6.486625,-7.363543,-8.509148,-11.173606,...,-7.483518,-7.928420,-6.892670,-6.053260,-6.119555,-5.187320,-9.118670,1,Corn,"POLYGON ((79.33320 18.22094, 79.33339 18.22107..."
4,-6.438655,-7.499561,-6.027222,-8.095627,-7.663611,-8.743983,-7.346660,-6.983235,-9.631377,-11.657347,...,-7.392450,-9.938508,-7.073982,-6.778762,-6.189435,-6.242713,-8.558163,1,Corn,"POLYGON ((79.33320 18.22094, 79.33339 18.22107..."


In [47]:
X_columns=trainingds.columns[0:-3]
X_columns

Index(['2022-12-30', '2023-01-11', '2023-01-23', '2023-02-04', '2023-02-16',
       '2023-02-28', '2023-03-12', '2023-03-24', '2023-04-05', '2023-04-17',
       '2023-04-29', '2023-05-11', '2023-05-23', '2023-06-04', '2023-06-16',
       '2023-06-28', '2023-07-10', '2023-07-22', '2023-08-03', '2023-08-15',
       '2023-08-27', '2023-09-08', '2023-09-20', '2023-10-02', '2023-10-14',
       '2023-10-26', '2023-11-07'],
      dtype='object')

In [48]:
X_columns=trainingds.columns[0:-3]
y_columns="Crop"
X_trainingds=trainingds[X_columns].to_numpy()
trainingds.loc[trainingds["Crop"]=="test", "class"]=0
trainingds.loc[trainingds["Crop"]=="Corn", "class"]=1
y_trainingds=trainingds[["class"]].to_numpy()
y_trainingds

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]])

In [49]:
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X_trainingds, y_trainingds, test_size=0.2,random_state=0)

In [50]:
X_train

array([[ -9.97359657, -11.37695122,  -7.34659624, ...,  -9.53531075,
         -9.12127304,  -9.45310211],
       [ -9.34912491,  -7.34021473,  -7.00150871, ...,  -7.64017487,
         -7.68250561,  -9.24023819],
       [-13.26651669,  -8.15732861,  -7.19911575, ...,  -7.26116085,
         -8.03079605,  -8.65097618],
       ...,
       [ -9.38765907,  -9.48782158,  -9.27001572, ...,  -6.44749403,
         -5.74987888,  -8.16541195],
       [ -5.6044054 ,  -8.48001957, -11.43142796, ...,  -5.58127308,
         -5.86533785,  -6.8823328 ],
       [ -9.19894886,  -9.18282223,  -8.83264637, ...,  -7.80822182,
         -8.58787727,  -5.47320175]])

In [51]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC

from sklearn.preprocessing import StandardScaler
from sklearn.kernel_approximation import PolynomialCountSketch

ps = PolynomialCountSketch(gamma=1.2, degree=3, n_components=270, random_state=1)
ps=ps.fit(X_train)

X_train_transformed = ps.transform(X_train)
X_test_transformed = ps.transform(X_test)

clf = make_pipeline(StandardScaler(), LinearSVC(max_iter=10000, random_state=0, tol=1e-5))

In [52]:
%%time
#Train the model using the training sets
clf.fit(X_train_transformed, y_train)

CPU times: user 8min 28s, sys: 129 ms, total: 8min 28s
Wall time: 8min 28s


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc',
                 LinearSVC(max_iter=10000, random_state=0, tol=1e-05))])

In [53]:
%%time
#Predict the response for test dataset
y_pred = clf.predict(X_test_transformed)

CPU times: user 127 ms, sys: 0 ns, total: 127 ms
Wall time: 46 ms


In [54]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

metrics.confusion_matrix(y_test, y_pred) 

array([[7058,  713],
       [ 665, 3916]])

In [55]:
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_train, y_pred))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_train, y_pred))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_train, y_pred))

ValueError: Found input variables with inconsistent numbers of samples: [49407, 12352]

In [179]:
import pickle
filekernel = 'kernel_poly.sav'
filename = 'linearsvc_model_poly.sav'
pickle.dump(ps, open(filekernel, 'wb'))
pickle.dump(clf, open(filename, 'wb'))